In [12]:
import re
#requests doesn't import here?
import requests
import json
from django.http import JsonResponse, HttpResponse, HttpResponseRedirect
link = "https://api.weather.gov/gridpoints/LIX/67,87/forecast"
#pull_Data.py should give some useful ideas for this


In [16]:
weather_dt = requests.get(link).json()
weather_data_str = f"{{\n\t\"weather_data\": {json.dumps(weather_dt)}\n}}"
print(weather_data_str)

{
	"weather_data": {"@context": ["https://geojson.org/geojson-ld/geojson-context.jsonld", {"@version": "1.1", "wx": "https://api.weather.gov/ontology#", "geo": "http://www.opengis.net/ont/geosparql#", "unit": "http://codes.wmo.int/common/unit/", "@vocab": "https://api.weather.gov/ontology#"}], "type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-90.1024706, 29.9332866], [-90.10341840000001, 29.9105538], [-90.0771818, 29.909729799999997], [-90.0762289, 29.932462499999996], [-90.1024706, 29.9332866]]]}, "properties": {"updated": "2024-03-27T02:11:32+00:00", "units": "us", "forecastGenerator": "BaselineForecastGenerator", "generatedAt": "2024-03-27T03:09:34+00:00", "updateTime": "2024-03-27T02:11:32+00:00", "validTimes": "2024-03-26T20:00:00+00:00/P7DT17H", "elevation": {"unitCode": "wmoUnit:m", "value": 4.8768}, "periods": [{"number": 1, "name": "Tonight", "startTime": "2024-03-26T22:00:00-05:00", "endTime": "2024-03-27T06:00:00-05:00", "isDaytime": false, "temperature":

In [17]:
def format_weather_data(periods):
    formatted_output = ""
    for period in periods:
        formatted_output += f"Name: {period['name']}\n"
        formatted_output += f"\tStart Time: {period['startTime']}\n"
        formatted_output += f"\tEnd Time: {period['endTime']}\n"
        formatted_output += f"\tTemperature: {period['temperature']} {period['temperatureUnit']}\n"
        formatted_output += f"\tShort Forecast: {period['shortForecast']}\n"
        formatted_output += f"\tDetailed Forecast: {period['detailedForecast']}\n\n"
    return formatted_output

In [18]:
# with open('https://api.weather.gov/gridpoints/LIX/67,87/forecast', 'r') as f:
#     weather_data = json.load(f)
response = requests.get(link)
if response.status_code == 200:
    #parse the json data from the response
    weather_data = response.json()
    # Extract and print relevant information
    periods = weather_data['properties']['periods']
    # for period in periods[:3]:  # Print data for the first three forecast periods, PLACEHOLDER SAMPLE
    #     print(f"Name: {period['name']}")
    #     print(f"Start Time: {period['startTime']}")
    #     print(f"End Time: {period['endTime']}")
    #     print(f"Temperature: {period['temperature']} {period['temperatureUnit']}")
    #     print(f"Short Forecast: {period['shortForecast']}")
    #     print(f"Detailed Forecast: {period['detailedForecast']}")
    #     print()
    formatted_output = format_weather_data(periods)
    print(formatted_output)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

Name: Tonight
	Start Time: 2024-03-26T22:00:00-05:00
	End Time: 2024-03-27T06:00:00-05:00
	Temperature: 55 F
	Short Forecast: Mostly Clear
	Detailed Forecast: Mostly clear. Low around 55, with temperatures rising to around 57 overnight. Northwest wind 0 to 10 mph.

Name: Wednesday
	Start Time: 2024-03-27T06:00:00-05:00
	End Time: 2024-03-27T18:00:00-05:00
	Temperature: 72 F
	Short Forecast: Sunny
	Detailed Forecast: Sunny. High near 72, with temperatures falling to around 70 in the afternoon. North wind 5 to 15 mph.

Name: Wednesday Night
	Start Time: 2024-03-27T18:00:00-05:00
	End Time: 2024-03-28T06:00:00-05:00
	Temperature: 52 F
	Short Forecast: Mostly Clear
	Detailed Forecast: Mostly clear, with a low around 52. North wind 5 to 10 mph.

Name: Thursday
	Start Time: 2024-03-28T06:00:00-05:00
	End Time: 2024-03-28T18:00:00-05:00
	Temperature: 71 F
	Short Forecast: Sunny
	Detailed Forecast: Sunny, with a high near 71. North wind 10 to 15 mph.

Name: Thursday Night
	Start Time: 2024-03-

#### Parades

In [3]:
import time
import requests
import json
import re
from datetime import datetime, timezone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

import datetime
import pickle
import os

import urllib3

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50, separators=['.', '\n'])
prepared_docs = []

    


In [4]:
with open('C:/Users/lmore/OneDrive/Documents/GitHub/Mardi-Gras-LLM/backend/backend/2025parades.json', 'r', encoding='utf-8') as f:
    r = f.read()
data = json.loads(r)

In [5]:


def add_prepared_doc(self, article):
    #This function converts the article retrieved as json article from the get request and converts it into an article, 
    #and stores it along with the other articles in the local class list "prepared_docs"

    # Extract necessary fields from each article
    uid = article.get("uuid", "")
    starttime = article.get("starttime", {}).get("utc", "") or "N/A"  # Use "N/A" if None
    title = article.get("title", "")
    url = article.get("url", "") or "N/A"
    updatetime = article.get("lastupdated", {}).get("utc", "") or "N/A"  # Use "N/A" if None
    page_content = article.get("content") or "N/A"
    
    if starttime == "N/A":
        starttime = 1    
    d2 = int((int(starttime) - 10)/1000)

    readable_starttime = 1
    if starttime == "N/A":
        readable_starttime = "N/A"
    else:
        readable_starttime = datetime.datetime.fromtimestamp(int(starttime)/1000).strftime("%A, %B %#d %Y")
    
    readable_updatetime = 1
    if updatetime == "N/A":
        readable_updatetime = "N/A"
    else:
        readable_updatetime = datetime.datetime.fromtimestamp(int(updatetime)/1000).strftime("%A, %B %#d %Y")

    if page_content != "N/A":
        cleaned_content = clean_html(page_content) 
        # Create a Document object with metadata
        doc = Document(page_content=cleaned_content, metadata={
            "uid": uid,
            "url": url,
            "utc_starttime": starttime,
            "readable_starttime": readable_starttime,
            "title": title,
            "utc_updatetime": updatetime,
            "readable_updatetime": readable_updatetime
        })
        prepared_docs.append(doc)
    
    return d2

    

In [6]:

def add_metadata(docs):
    
    #print(datetime.datetime.fromtimestamp(int(docs[0].metadata["starttime"])/1000,tz=datetime.timezone.utc).isoformat())

    for doc_number, doc in enumerate(docs):
        starttime = docs[doc_number].metadata["starttime"]
        updatetime = docs[doc_number].metadata["updatetime"]

        if starttime != "N/A":
            docs[doc_number].metadata["readable_starttime"] = datetime.datetime.fromtimestamp(int(starttime)/1000).strftime("%A, %B %#d %Y")
        else:
            docs[doc_number].metadata["readable_starttime"] = "N/A"
        
        docs[doc_number].metadata["utc_starttime"] = starttime
        del docs[doc_number].metadata["starttime"]

        if updatetime != "N/A":
            docs[doc_number].metadata["readable_updatetime"] = datetime.datetime.fromtimestamp(int(updatetime)/1000).strftime("%A, %B %#d %Y")
        else:
            docs[doc_number].metadata["readable_updatetime"] = "N/A"
        
        docs[doc_number].metadata["utc_updatetime"] = updatetime
        del docs[doc_number].metadata["updatetime"]

    return docs

In [7]:
def clean_html( raw_html):
    """
    Remove HTML tags and unwanted content from a string or list of strings.
    :param raw_html: A string or list of strings containing HTML content.
    :return: Cleaned string or list of cleaned strings.
    """
    for line in raw_html:
        if "inline-asset inline-editorial-image" in line:
            try:
                image_link = line[line.index("https://bloximages.newyork1.vip.townnews.com/nola.com/content"):line.index(".image.jpg")+10]
                description = line[line.index("<p>"):line.index("</p>")]
                new_line = "JPG Image URL: " + image_link + " " + description
                raw_html.remove(line)
                raw_html.append(new_line)
            except:
                pass
        elif "<figure" in line:
            raw_html.remove(line)

    if isinstance(raw_html, list):
        raw_html = ' '.join(raw_html)

    clean_text = re.sub(r'<[^>]+>', '', raw_html)  # Remove all remaining HTML tags

    return clean_text
    

In [8]:
html = data['content']
html_text = " ".join(html)

In [13]:
import re

def clean_and_structure_html(html):
    structured_data = {}
    current_parade = {}
    
    first_line = html[0]
    year_match = re.search(r'20\d{2}', first_line)
    year_match = year_match.group(0) if year_match else None
    print(year_match)
    
    for line in html:
        # Check for parade title
        if '<h1><strong' in line:
            # Save current parade if exists
            if current_parade and 'parade_name' in current_parade:
                structured_data[current_parade['parade_name']] = current_parade
            current_parade = {}
            # Extract and set the parade name
            current_parade['parade_name'] = re.search('<h1><strong[^>]*>(.*?)</strong></h1>', line).group(1)

        # Check for date and location
        elif '<p><strong>' in line:
            # Remove HTML tags for easier processing
            clean_line = re.sub('<[^>]*>', '', line)
            # Attempt to extract date, time, and location with a more flexible approach
            date_time_location = re.match(r'(.*?),(.*?) (a.m.|p.m.|morning), (.*)', clean_line)
            if date_time_location:
                current_parade['date'] = f"{date_time_location.group(1).strip()}, {date_time_location.group(2).strip()} {date_time_location.group(3).strip()} {year_match}"
                current_parade['location'] = date_time_location.group(4).strip()


        # Check for image
        elif 'inline-asset inline-editorial-image' in line:
            image_url_match = re.search('src="(.*?)"', line).group(1)
            # Remove "?resize" and anything that follows
            clean_image_url = re.sub(r'\?.*$', '', image_url_match)
            current_parade['parade_route'] = clean_image_url

        # Check for description and other details
        elif '<p>' in line and not 'For more information' in line:
            description = re.search('<p>(.*?)</p>', line).group(1)
            if 'description' in current_parade:
                current_parade['description'] += ' ' + description
            else:
                current_parade['description'] = description

        # Check for website URL
        elif 'For more information' in line:
            current_parade['website'] = re.search('href="(.*?)"', line).group(1)

    # Add the last parade to the list
    if current_parade and 'parade_name' in current_parade:
        structured_data[current_parade['parade_name']] = current_parade


    return structured_data

In [14]:
import json
parade_info = clean_and_structure_html(html)
# Convert parade_info to JSON string

parade_info_json = json.dumps(parade_info)

# Write parade_info_json to a file
with open('parade_info.json', 'w') as f:
    f.write(parade_info_json)


2025
